## Imports

In [ ]:
import math
# import time
import numpy as np
import pandas as pd
import Modules.Geometry as geo
# from io import StringIO
from ansys.mapdl import reader as pymapdl_reader
from ansys.mapdl.reader import examples
from ansys.mapdl.core import Mapdl
from ansys.mapdl.core import launch_mapdl


## Constants and Launch MAPDL

In [ ]:
PI = math.pi

kdictD = geo.kdictD
kdictWeld = geo.kdictWeld
R = geo.R

Emodul_1 = 210000
Emodul_2 = 210000
Possionsratio = 0.3

In [ ]:
mapdl = launch_mapdl(override=True, loglevel='ERROR')

## Generating Geometry

### General Geometry

In [ ]:
mapdl.clear()
mapdl.prep7()

mapdl.esize(geo.esizedefault)
mapdl.et('1', 'solid187')

for key in kdictD.keys():
    mapdl.k(key, kdictD[key][0], kdictD[key][1], kdictD[key][2])

mapdl.larc(3, 5, 41, R)  # mapdl.larc(Punkt1, Punkt2,
mapdl.larc(4, 6, 42, R)             # Mittelpunkt, Radius)
mapdl.larc(37, 35, 47, R)
mapdl.larc(38, 36, 48, R)
mapdl.larc(15, 17,  43, R)
mapdl.larc(18, 16,  44, R)
mapdl.larc(23, 25, 45, R)
mapdl.larc(26, 24, 46, R)

# Volumen - 1 - Einspannbacken links
mapdl.v(1, 81, 87, 39, 2, 82, 88, 40)
# Volumen - 2 - Anschlusss-Einspannbacken-Rundung
mapdl.v(81, 3, 37, 87, 82, 4, 38, 88)
# Volumen - 3 - links - Rundung
mapdl.v(3, 5, 35, 37, 4, 6, 36, 38)
# Volumen - 4 - Anschluss - SSK
mapdl.v(5, 65, 79, 35, 6, 66, 80, 36)
# Volumen - 5 - SSK
mapdl.v(65, 67, 77, 79, 66, 68, 78, 80)
# Volumen - 6 - SSK-Schweissnaht
mapdl.v(67, 50, 62, 77, 68, 51, 63, 78)
# Volumen - 7 - Schweissnaht-Steg
mapdl.v(50, 8, 32, 62, 51, 9, 33, 63)
# Volumen - 8 - Schweissnähte -  links oben
mapdl.v(33, 64, 63, 9, 52, 51)
# Volumen - 9 - Schweissnähte -  links unten
mapdl.v(32, 62, 61, 8, 50, 49)

# VOLUMEN VON BLECH3
# Volumen - 10 - Mittelblech: +z
mapdl.v(7, 11, 27, 31, 49, 53, 57, 61)
# Volumen - 11 - Mittelblech: Schweißnaht +z
mapdl.v(49, 53, 57, 61, 8, 12, 28, 32)
# Volumen - 12 - Mittelblech: z=0
mapdl.v(8, 12, 28, 32, 9, 13, 29, 33)
# Volumen - 13 - Mittelblech: Schweißnaht -z
mapdl.v(9, 13, 29, 33, 52, 56, 60, 64)
# Volumen - 14 - Mittelblech: -z
mapdl.v(52, 56, 60, 64, 10, 14, 30, 34)

# VOLUMEN VON BLECH2
# Volumen - 15 - Schweißnaht-Steg
mapdl.v(12, 54, 58, 28, 13, 55, 59, 29)
# Volumen - 16 - SSK-Schweißnaht
mapdl.v(54, 69, 75, 58, 55, 70, 76, 59)
# Volumen - 17 - SSK
mapdl.v(69, 71, 73, 75, 70, 72, 74, 76)
# Volumen - 18 -Anschluss - SSK
mapdl.v(71, 15, 25, 73, 72, 16, 26, 74)

# Volumen - 19 -Schweißnähte -  rechts oben
mapdl.v(29, 60, 59, 13, 56, 55)
# Volumen - 20 - Schweißnähte -  rechts unten
mapdl.v(28, 58, 57, 12, 54, 53)

# Volumen - 21 - rechts - Rundung
mapdl.v(15, 17, 23, 25, 16, 18, 24, 26)
# Volumen - 22 - Anschluss Einspannbacken - Rundung
mapdl.v(17, 83, 85, 23, 18, 84, 86, 24)
# Volumen - 23 - Einspannbacken rechts
mapdl.v(83, 19, 21, 85, 84, 20, 22, 86)

# mapdl.vglue("ALL")
mapdl.vclear('all')  # Löscht bisherige Vernetzung

mapdl.allsel('all')
mapdl.cm('abksk', 'volu')

## No Loops, all volumes in one go

### Inner Cylinder

In [ ]:
# Introductory Notes:
# "abksk" heisst "all but ksk" und meint alle Volumen ausser die Ausrundung vor der Schweissnaht des Kerbspannungskonzeptes.

kskstring1 = 'kskinner'
kskstring2 = 'kskouter'

# Erstellung der 4 Cylinder vor der Schweissnaht.
mapdl.wpcsys(1, 0)
mapdl.kwpave(91)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r2, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(92)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r2, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(93)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r2, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(94)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r2, 0, geo.W2)

# Anwaehlen der 4 Cylinder
mapdl.vsel('s', 'loc', 'x', kdictD[91][0])
mapdl.vsel('a', 'loc', 'x', kdictD[93][0])
mapdl.cm(kskstring1, 'volu')
mapdl.allsel('all')
# Die Cylinder werden durch den urspruenglichen Kreuzstoss geteilt aber alle Volumen bleiben dabei erhalten durch keep
mapdl.vsbv(kskstring1, 'all', '', 'keep', 'keep')

# Anwaehlen der 4 Cylinder und abschneiden vom "oberen" Teil
mapdl.vsel('s', 'loc', 'x', kdictD[91][0])
mapdl.vsel('a', 'loc', 'x', kdictD[93][0])
mapdl.vsbv(kskstring1, 'all')
mapdl.allsel('below', 'volu')

# Wir muessen die Volumen miteinander verbinden, da durch die vorheringen Operationen zwei einzelne entstanden sind
mapdl.lsel('r', 'loc', 'x', kdictD[91][0])
mapdl.lsel('a', 'loc', 'x', kdictD[93][0])
mapdl.asll('s')
mapdl.vsla('s')
mapdl.vadd('all')
mapdl.cm(kskstring1, 'volu')

# Selbiges mit den Flaechen
mapdl.aslv('s')
mapdl.asel('r', 'loc', 'y', kdictD[91][1])
mapdl.aadd('all')
mapdl.aslv('s')
mapdl.asel('r', 'loc', 'y', -geo.W2/2)
mapdl.aadd('all')
mapdl.allsel('all')
mapdl.cmsel('all')

# Ausschneiden des Volumens vor der Schweissnaht und gleichzeitig behalten wir das Volumen der Ausrundung.
mapdl.vsbv('abksk', kskstring1, '', '', 'keep')

""" ANSYS verhaelt sich manchmal extrem komisch, deshalb muessen wir die naechsten Schritte etwas kompliziert ausfuehren:
Zuerst waehlen wir die zwei Volumen des horizontalen Blechs vor der Schweissnaht jeweils rechts und links.
Anschliessend waehlen wir die Schweissnaehte aus und fuegen alles zu einer Gruppe "delegrp" zusammen,
welche dann von den oben erstellten Cylindern ausgeschnitten werden. Die Cylinder behalten wir mit 'keep'."""

mapdl.asel('s', 'loc', 'x', kdictD[67][0])
mapdl.vsla('s')
mapdl.cm('HSR', 'volu')
mapdl.allsel('all')

mapdl.asel('s', 'loc', 'x', kdictD[69][0])
mapdl.vsla('s')
mapdl.cm('HSL', 'volu')
mapdl.allsel('all')

mapdl.lsel('s', 'loc', 'z', kdictD[52][2])
mapdl.lsel('a', 'loc', 'z', kdictD[49][2])
mapdl.lsel('u', 'loc', 'y', kdictD[50][1])
mapdl.lsel('u', 'loc', 'y', kdictD[62][1])
mapdl.asll('s')
mapdl.asel('u', 'loc', 'x', kdictD[49][0])
mapdl.asel('u', 'loc', 'x', kdictD[53][0])
mapdl.asel('u', 'loc', 'z', kdictD[49][2])
mapdl.asel('u', 'loc', 'z', kdictD[52][2])
mapdl.vsla('s')
mapdl.cm('sn', 'volu')
mapdl.allsel('all')

mapdl.cmgrp('delegrp', 'HSR', 'HSL', 'sn')
mapdl.vsbv('delegrp', kskstring1, '', '', 'keep')
mapdl.allsel('all')

### Outer Cylinder

In [ ]:
mapdl.wpcsys(1, 0)
mapdl.kwpave(91)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r2, geo.r3, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(92)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r2, geo.r3, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(93)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r2, geo.r3, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(94)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r2, geo.r3, 0, geo.W2)

mapdl.vsel('s', 'loc', 'x', kdictD[91][0])
mapdl.vsel('a', 'loc', 'x', kdictD[93][0])
mapdl.cm(kskstring2, 'volu')
mapdl.allsel('all')
mapdl.vsbv(kskstring2, 'all', '', 'keep', 'keep')
mapdl.allsel('all')

mapdl.vsel('s', 'loc', 'x', kdictD[91][0])
mapdl.vsel('a', 'loc', 'x', kdictD[93][0])
tempstring = 'ksk2'
mapdl.cm(tempstring, 'volu')
mapdl.allsel('all')

mapdl.lsel('s', 'radius', '', geo.r3)
mapdl.asll('s')
mapdl.vsla('s')
mapdl.cmsel('u', tempstring)
mapdl.cm('delevolu', 'volu')
mapdl.cmsel('a', tempstring)
mapdl.vsbv(tempstring, 'delevolu')
mapdl.cm(kskstring2, 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

### Reselecting all volumes and glue them together

In [ ]:
# Volumes on the left side (former number 6 and 7)

mapdl.lsel('s', 'loc', 'x', kdictD[67][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllone', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.lsel('s', 'loc', 'x', kdictD[51][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('blltwo', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

# Volumes on the left side (former number 15 and 16)

mapdl.lsel('s', 'loc', 'x', kdictD[69][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllthree', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.lsel('s', 'loc', 'x', kdictD[55][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllfour', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

# Anwaehlen der Schweissnaehte
mapdl.lsel('s', 'loc', 'z', kdictD[52][2])
mapdl.lsel('a', 'loc', 'z', kdictD[49][2])
mapdl.lsel('u', 'loc', 'y', kdictD[52][1])
mapdl.lsel('u', 'loc', 'y', kdictD[64][1])
mapdl.asll('s')
mapdl.asel('u', 'loc', 'x', kdictD[52][0])
mapdl.asel('u', 'loc', 'z', kdictD[52][2])
mapdl.asel('u', 'loc', 'x', kdictD[56][0])
mapdl.asel('u', 'loc', 'z', kdictD[49][2])
mapdl.vsla('s')
mapdl.cm('sn', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.vsbv('abksk', kskstring2, '', '', 'keep')
mapdl.allsel('all')
mapdl.vglue('all')

### Reselecting volumes again

In [ ]:
# Volumes on the left side (former number 6 and 7)

mapdl.lsel('s', 'loc', 'x', kdictD[67][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllone', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.lsel('s', 'loc', 'x', kdictD[51][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('blltwo', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

# Volumes on the left side (former number 15 and 16)

mapdl.lsel('s', 'loc', 'x', kdictD[69][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllthree', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.lsel('s', 'loc', 'x', kdictD[55][0])
mapdl.asll('s', '1')
mapdl.vsla('s')
mapdl.cmsel('u', 'abksk')
mapdl.cm('bllfour', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

# Anwaehlen der Schweissnaehte

mapdl.lsel('s', 'loc', 'z', kdictD[52][2])
mapdl.lsel('a', 'loc', 'z', kdictD[49][2])
mapdl.lsel('u', 'loc', 'y', kdictD[52][1])
mapdl.lsel('u', 'loc', 'y', kdictD[64][1])
mapdl.asll('s')
mapdl.asel('u', 'loc', 'x', kdictD[52][0])
mapdl.asel('u', 'loc', 'z', kdictD[52][2])
mapdl.asel('u', 'loc', 'x', kdictD[56][0])
mapdl.asel('u', 'loc', 'z', kdictD[49][2])
mapdl.vsla('s')
mapdl.cm('sn', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.allsel('all')
mapdl.cmsel('all')

### Radius at the cross sheet

In [ ]:
rblqstring1 = 'roundingblq'

mapdl.wpcsys(1, 0)
mapdl.kwpave(95)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r4, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(96)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r4, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(97)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r4, 0, geo.W2)

mapdl.wpcsys(1, 0)
mapdl.kwpave(98)
mapdl.wprota(0, 90, 0)
mapdl.cylind(geo.r1, geo.r4, 0, geo.W2)

mapdl.vsel('s', 'loc', 'x', kdictD[95][0])
mapdl.vsel('a', 'loc', 'x', kdictD[97][0])
mapdl.cm(rblqstring1, 'volu')
mapdl.allsel('all')
mapdl.vsbv(rblqstring1, 'all', '', 'keep', 'keep')

mapdl.lsel('s', 'radius', '', geo.r4)
mapdl.asll('s')
mapdl.vsla('s')
mapdl.vsel('u', 'volu', '', rblqstring1)
mapdl.cm('rblqdele', 'volu')
mapdl.vsel('a', 'volu', '', rblqstring1)
mapdl.vsbv(rblqstring1, 'rblqdele')
mapdl.allsel('below', 'volu')

mapdl.lsel('r', 'loc', 'z', kdictD[95][2])
mapdl.lsel('a', 'loc', 'z', kdictD[96][2])
mapdl.asll('s')
mapdl.vsla('s')
mapdl.vadd('all')
mapdl.cm(rblqstring1, 'volu')

mapdl.aslv('s')
mapdl.asel('r', 'loc', 'y', kdictD[95][1])
mapdl.aadd('all')
mapdl.aslv('s')
mapdl.asel('r', 'loc', 'y', -geo.W2/2)
mapdl.aadd('all')
mapdl.allsel('all')
mapdl.cmsel('all')

mapdl.vsbv('abksk', rblqstring1, '', '', 'keep')
mapdl.allsel('all')
mapdl.vglue('all')

## Reselecting Volumes

### Reselect cross sheet

In [ ]:
mapdl.vsel('s', 'loc', 'x', 0)
mapdl.cm('blq', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.cmsel('all')
mapdl.allsel('all')

### Reselect rounding at cross sheet

In [ ]:
mapdl.lsel('s', 'radius', '', geo.r1)
mapdl.asll('s')
mapdl.vsla('s')
mapdl.cmsel('u', kskstring1)
mapdl.cm(rblqstring1, 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.cmsel('all')
mapdl.allsel('all')

### Reselect weldings

In [ ]:
mapdl.lsel('s', 'radius', '', geo.r4)
mapdl.asll('s')
mapdl.vsla('s')
mapdl.cmsel('u', 'blq')
mapdl.cmsel('u', rblqstring1)
mapdl.cm('sn', 'volu')
mapdl.cmsel('a', 'abksk')
mapdl.cm('abksk', 'volu')
mapdl.cmsel('all')
mapdl.allsel('all')

### Reselect outer ksk rounding

In [ ]:
mapdl.cmsel('u', 'abksk')
mapdl.lsel('s', 'radius' '', geo.r3)
mapdl.asll('s')
mapdl.vsla('r')
mapdl.cm(kskstring2, 'volu')
mapdl.cmgrp('kskges', kskstring1, kskstring2)
mapdl.cmsel('all')
mapdl.allsel('all')

## Meshing

### Meshing inner part of rounding

In [ ]:
mapdl.cmsel('s', kskstring1)
mapdl.allsel('below', 'volu')
mapdl.kesize('all', geo.esizeksk)
mapdl.lesize('all', geo.esizeksk)
mapdl.aesize('all', geo.esizeksk)
mapdl.vmesh('all')
mapdl.allsel('all')
mapdl.cmsel('all')

### Meshing outer part of rounding

In [ ]:
mapdl.cmsel('s', kskstring2)
mapdl.allsel('below', 'volu')
mapdl.kesize('all', geo.esize2)
mapdl.lesize('all', geo.esize2)
mapdl.aesize('all', geo.esize2)
mapdl.vmesh('all')
mapdl.allsel('all')
mapdl.cmsel('all')

### Coarser mesh after 0.4 * T

In [ ]:
mapdl.esize(1)
mapdl.asel('s', 'loc', 'x', kdictD[68][0])
mapdl.asel('a', 'loc', 'x', kdictD[70][0])
mapdl.allsel('below', 'area')
mapdl.kesize('all', 2)
mapdl.lesize('all', 2)
mapdl.cmsel('s', 'bllone')
mapdl.cmsel('a', 'bllthree')
mapdl.vmesh('all')
mapdl.allsel('all')
mapdl.cmsel('all')

In [ ]:
mapdl.eplot(notebook=False)

### Coarser mesh after 1.0 * T

In [ ]:
mapdl.asel('s', 'loc', 'x', kdictD[66][0])
mapdl.asel('s', 'loc', 'x', kdictD[72][0])
mapdl.allsel('below', 'area')
mapdl.kesize('all', 5)
mapdl.lesize('all', 5)
mapdl.aesize('all', 5)
mapdl.allsel('all')

### Meshing of middle volume in horizontal sheet between two welds

In [ ]:
mapdl.esize(1)
mapdl.cmsel('s', 'blltwo')
mapdl.cmsel('a', 'bllfour')
mapdl.allsel('below', 'volu')
mapdl.kesize('all')
mapdl.lesize('all')
mapdl.aesize('all')
mapdl.vmesh('all')
mapdl.allsel('all')
mapdl.cmsel('all')

### Meshing remaining volumes

In [ ]:
# mapdl.cmsel('s', 'abksk')
# mapdl.allsel('below', 'volu')
# mapdl.kesize('all', geo.esizedefault)
# mapdl.lesize('all', geo.esizedefault)
# mapdl.aesize('all', geo.esizedefault)
mapdl.esize(geo.esizedefault/2)
mapdl.vmesh(4, 5)
mapdl.vmesh(17, 18)

In [ ]:
mapdl.esize(geo.esizedefault)
mapdl.vmesh(1, 3)
mapdl.vmesh(21, 23)
# mapdl.eplot(notebook=False)

## Exit Mapdl

In [ ]:
mapdl.exit()